In [26]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [ ]:
# portfolio of stocks
# stocks = ['VICI', 'JOE', 'ELS', 'IIPR-PA', 'G24.DE'] # real_estate
stocks = ['RACE', 'SMCI', 'CRM', 'DIS', 'ACN', 'PINS', 'NVO', 'ADBE', 'LULU', 'PYPL', 'MU'] # carbon finance
# stocks = ['2897.T', 'CLX', 'NATR', 'GRUMAB.MX', 'CMG', 'ADS.DE', 'LULU', 'DECK'] # consumer_discretion 
# stocks = ['RACE', 'LVS', 'MRK', 'NIO', 'PYPL', 'ADBE', 'FTNT', 'CL', 'ZTS', 'LDOS', 'MNST', 'ADSK', 'MDLZ', 'ISRG', 'MRK', 'NVO', 'UNH', '4528.T', '4568.T', 'BRO', 'DEMANT.CO', 'HRMY', 'HSTM', 'TWLO', 'CRM', 'DUOL', 'GDDY', 'ACN', 'TPL', 'DOCU']

#### Download historical data from yf API

In [28]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# download data to csv
# data.to_csv("stocks.csv")
# data.to_csv("stocks_real_estate.csv")
data.to_csv("stocks_consumer_discretion.csv")

[*********************100%***********************]  8 of 8 completed


#### Download analysis from csv downloaded data

In [29]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
# df = pd.read_csv("stocks.csv", header=[0,1], index_col=0)
df = pd.read_csv("stocks_consumer_discretion.csv", header=[0,1], index_col=0)
# df = pd.read_csv("stocks_real_estate.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()

Ticker           2897.T                                                   \
Price              Open         High          Low        Close    Volume   
Date                                                                       
2020-01-02          NaN          NaN          NaN          NaN       NaN   
2020-01-03          NaN          NaN          NaN          NaN       NaN   
2020-01-06  2436.863606  2445.967704  2412.585938  2412.585938  642900.0   
2020-01-07  2415.620728  2473.280166  2415.620728  2467.210693  471600.0   
2020-01-08  2427.759671  2439.898395  2403.482001  2436.863770  508200.0   

Ticker            CMG                                               ...  \
Price            Open       High        Low      Close      Volume  ...   
Date                                                                ...   
2020-01-02  16.799400  17.183800  16.750601  17.163799  24365000.0  ...   
2020-01-03  17.064800  17.418800  17.064800  17.302601  21135000.0  ...   
2020-01-06  17.240000  17.256001  17.080000  17.160400  20260000.0  ...   
2020-01-07  17.260000  17.260000  17.087799  17.201599  12165000.0  ...   
2020-01-08  17.122999  17.340000  17.122999  17.134600  13125000.0  ...   

Ticker       GRUMAB.MX                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2020-01-02  174.394380  176.012772  173.540733  174.572235  351684.0   
2020-01-03  173.665232  177.728973  173.665232  176.243973  391018.0   
2020-01-06  175.808251  176.066120  173.834176  174.065369  404230.0   
2020-01-07  174.020921  174.465532  171.175410  172.135773  592764.0   
2020-01-08  172.272197  176.226306  172.272197  175.904251  669038.0   

Ticker            LULU                                                 
Price             Open        High         Low       Close     Volume  
Date                                                                   
2020-01-02  232.899994  233.470001  231.770004  233.419998  1449300.0  
2020-01-03  231.240005  234.500000  230.229996  232.639999  1315400.0  
2020-01-06  231.490005  235.529999  230.800003  235.429993  1460700.0  
2020-01-07  235.500000  236.850006  233.339996  233.699997  1367900.0  
2020-01-08  234.880005  238.009995  234.009995  237.270004  1883900.0  

[5 rows x 40 columns]

In [30]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All Stocks

In [31]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for stock in stocks:
    # Use the stock data from our df
    df_stock = df[stock][['Close', 'Volume']]

    # Convert the stock to datetime if needed
    if not isinstance(df_stock.index, pd.DatetimeIndex):
        df_stock.index = pd.to_datetime(df_stock.index)

    # plot
    hist = go.Histogram(
        y=df_stock['Close'],
        x=df_stock['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {stock} Close Prices'
    )

    fig.show()